In [ ]:
import sys
# add parent directory to sys.path so that python finds the modules
sys.path.append('..')

import pandas as pd
import scipy.stats
import numpy as np
from matplotlib import rc
from matplotlib import pyplot as plt

from db_utils import DatabaseConnection

In [6]:
with DatabaseConnection() as cur:
    cur.execute("""
        select seg_id, max(accel), sum(duration), sum(dist)
        from accels 
        where type='a' 
        group by seg_id, filename 
        having max(accel)<2 and sum(dist)<350 and sum(dist)>20 and min(velo)=0
    """)
    res = cur.fetchall()
df = pd.DataFrame(res, columns=['seg_id', 'max_a', 'duration', 'distance'])

In [8]:
# Normal distribution

SUMO_DEFAULT_VALUE = 1.2

plt.figure(figsize=(8,8))
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': 16})
rc('text', usetex=True)
plt.xlabel(r'$a_{max}$ in m/s²')
plt.ylabel('Relative frequency')

plt.hist(df.max_a, bins=100, density=True, alpha=0.5, label=r'$a_{max}^{SimRa}$')

plt.vlines(SUMO_DEFAULT_VALUE, 0, 4, colors='green', linewidth=3, label=r'$a_{max}^{SUMO}$')

mean, std = scipy.stats.norm.fit(df.max_a)
x = np.linspace(0, 3, 1000)
y = scipy.stats.norm.pdf(x, mean, std)
plt.plot(x, y, color='orange', linewidth=3, label="$\mathcal{N}$" + f"$({mean:.2f}, {std:.2f})$", alpha=0.5, zorder=2)
plt.fill_between(x, y, [0] * len(x), color='orange', alpha=0.5, zorder=2)


plt.xlim(0, 2.2)
plt.ylim(0, 1.8)

plt.title("Maximum Acceleration")
plt.legend()
plt.savefig("max_accel_analysis.png", dpi=300, bbox_inches='tight')

print(f"Mean: {mean}, Std: {std}")
print(f"Median: {df.max_a.median()}")

print(f"Lower cutoff (0.05 percentile): {scipy.stats.norm.ppf(0.05, mean, std)}")
print(f"Upper cutoff (0.95 percentile): {scipy.stats.norm.ppf(0.95, mean, std)}")

print(f"CDF at SUMO default value: {scipy.stats.norm(mean, std).cdf(SUMO_DEFAULT_VALUE)}")

RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 576x576 with 1 Axes>

In [ ]:
# plot representative Acc maneuvers

from scipy.interpolate import make_interp_spline, BSpline

def plot_accels(x, one_plot):
    
    x['cum_duration'] = x.duration.cumsum()

    x = x[~x.a.isnull()]

    xnew = np.linspace(x.cum_duration.min(), x.cum_duration.max(), 1000) 
    try:
        spl_a = make_interp_spline(x.cum_duration, x.a)
        smooth_a = spl_a(xnew)
    except: 
        return
    
    plt.figure(figsize=(7,7))

    plt.plot(xnew, smooth_a, label='Acceleration profile')
    plt.scatter(x.cum_duration, x.a)

    plt.xlabel('Duration in seconds')
    plt.ylabel('Acceleration in m/s²')
    plt.title(f"{list(x.filename)[0]} {list(x.seg_id)[0]}")
    plt.legend()

    #plt.plot(xnew, smooth_v)
    #plt.scatter(x.cum_duration, x.velo)
    # plt.savefig('images/acc_processes/%i.png'%x.seg_id.values[0],dpi=300, bbox_inches='tight')
    plt.show()
    

with DatabaseConnection() as cur:
    cur.execute("""select filename, seg_id, accel, duration, velo from accels where type = 'a' Limit 50""")
    res = cur.fetchall()
    df = pd.DataFrame(res, columns=['filename', 'seg_id','a', 'duration', 'velo'])
    display(df)
    df.groupby(['seg_id', 'filename'], as_index=False).apply(lambda x: plot_accels(x, False))